In [1]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

In [2]:
cd /home/jupyter/

/home/jupyter


In [3]:
df_train = pd.read_parquet('intellizenz-model-training/data/export_training_features_2016_2020.parquet.gzip')
df_test = pd.read_parquet('intellizenz-model-training/data/export_testing_features_2016_2020.parquet.gzip')

Build a XGBoost model to classify event segments

In [4]:
X_train = df_train.loc[:,~df_train.columns.isin(['veranst_segment','vg_inkasso'])] # 152 features
y_train = df_train['veranst_segment']

X_test = df_test.loc[:,~df_test.columns.isin(['veranst_segment','vg_inkasso'])] # 152 features
y_test = df_test['veranst_segment']

In [5]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [6]:
xgboost_classifier = xgboost.XGBClassifier(n_estimators=500, max_depth=7, use_label_encoder=False, objective='binary:logistic',eval_metric = 'error')

In [7]:
# xgboost_classifier = xgboost.XGBClassifier()
xgboost_classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='error', gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=7, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=500,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [9]:
# save the model to disk
xbg_filename = 'intellizenz-model-training/models/2022-10-06/xgb_classifier_model_with_inkasso_time_series.sav'
pickle.dump(xgboost_classifier, open(xbg_filename, 'wb'))

In [10]:
y_pred = xgboost_classifier.predict(X_test)

Evaluation

In [11]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

xgboost_accuracy = accuracy_score(y_test, y_pred, normalize=False)
print(xgboost_accuracy)

[[115313   8400  19289]
 [ 35817   8481  17379]
 [ 45464   8698  89608]]
              precision    recall  f1-score   support

           0       0.59      0.81      0.68    143002
           1       0.33      0.14      0.19     61677
           2       0.71      0.62      0.66    143770

    accuracy                           0.61    348449
   macro avg       0.54      0.52      0.51    348449
weighted avg       0.59      0.61      0.59    348449

213402
